# Equipment_move_workflow

# IT_Equipment_unique_records

Workflow permettant la mise à jour de la table `IT_equipments_unique_equipments`

`IT_Equipment_sorted_by_date_asc` : table ordonnée par date croissante 

`IT_Equipment_sorted_by_date_desc` : table ordonnée par date décroissante

In [ ]:
drop view if exists IT_Equipment_sorted_by_date_asc ;
create view `IT_Equipment_sorted_by_date_asc` 
as
select ASSET_ID, ID_Systeme_Asset, Modele, Nom_de_la_salle, Reperage_ID, SOURCES, INSERTED from `IT_Equipment_records` 
ORDER BY STR_TO_DATE((inserted),'%y-%m-%d') ASC ;

drop view if exists IT_Equipment_sorted_by_date_desc ;
create view `IT_Equipment_sorted_by_date_desc` 
as
select ASSET_ID, ID_Systeme_Asset, Modele, Nom_de_la_salle, Reperage_ID, SOURCES, INSERTED from `IT_Equipment_records` 
ORDER BY STR_TO_DATE((inserted),'%y-%m-%d') DESC ;

`IT_Equipment_first_last_occurence_per_id_all` : 1er occurence et dernière occurence pour chaque asset_id

In [ ]:
drop view if exists IT_Equipment_first_last_occurence_per_id_all ;
CREATE view IT_Equipment_first_last_occurence_per_id_all
as 
select min(inserted) first_records, max(inserted) last_records, it.ID_Systeme_Asset from `IT_Equipment_sorted_by_date_asc` it
GROUP BY ID_Systeme_Asset
order by INSERTED desc ;

`IT_Equipment_first_occurence_per_id` : Première occurence (date d'ajout d'un equipment) 

`IT_Equipment_last_occurence_per_id` : Dernière occurence (dernière date d'update d'un equipment) 

In [ ]:
drop view if exists IT_Equipment_first_occurence_per_id ;
CREATE view  IT_Equipment_first_occurence_per_id
as
select it.* from `IT_Equipment_records` it
INNER JOIN `IT_Equipment_first_last_occurence_per_id_all` it2 
ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset
AND it.inserted = it2.first_records ;

drop view if exists IT_Equipment_last_occurence_per_id ;
CREATE view  IT_Equipment_last_occurence_per_id
as
select it.* from `IT_Equipment_records` it
INNER JOIN `IT_Equipment_first_last_occurence_per_id_all` it2 
ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset
AND it.inserted = it2.last_records ;

`IT_Equipment_last_occurence_position` : Asset, salle, position et date d'insertion de la dernière position connu d'un equipment. Exemple : changement de position de 22-06-01, et mise à jour le  22-06-03 & 22-06-10. La première fois que la nouvelle position apparait est affiché (donc 22-06-01)

In [ ]:
drop view if exists IT_Equipment_last_occurence_position ;
CREATE view  IT_Equipment_last_occurence_position
as
SELECT it2.ID_Systeme_Asset, it2.nom_de_la_salle, it2.Reperage_ID, it2.INSERTED
FROM `IT_Equipment_last_occurence_per_id` it
INNER JOIN `IT_Equipment_records` it2 ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset AND it.reperage_id = it2.reperage_id
GROUP BY it2.ID_Systeme_Asset

`last_date_insertion` : dernière date du fichier csv de mise à jour de la base. Exemple IT_equipment.01.06.22 -> 22-06-01

In [ ]:
drop view if EXISTS last_date_insertion ;
create view last_date_insertion
as
select DISTINCT(INSERTED) source_insert from `IT_Equipment_records`
ORDER by INSERTED desc
limit 1

`date_insertion_records` : table historique de toutes les mise à jour de la base

In [ ]:
drop view if EXISTS date_insertion_records ; 
create view date_insertion_records
as
select DISTINCT(INSERTED) source_insert from `IT_Equipment_records`
ORDER by INSERTED desc ;

`IT_Equipment_cleaned_equipment` : Table des equipments clean, avec leur asset_id, leur dernière position connue et la dernière date d'apparition dans un fichier csv. Un Equipement est jugé clean s'il n'apparait pas dans la dernière MàJ de la base

In [ ]:
drop view if EXISTS IT_Equipment_cleaned_equipment;
create view IT_Equipment_cleaned_equipment
as 
select it.nom, it.ID_Systeme_Asset, it.asset_id, it.nom_de_la_salle, it.Reperage_ID,it.SOURCES, it.INSERTED last_insert_date  from `IT_Equipment_last_occurence_per_id` it
LEFT JOIN `last_date_insertion` dat ON dat.source_insert = it.inserted
where dat.source_insert is null ;

`IT_Equipment_unique_records` : table regroupant tous les asset_id unique, une ligne par equipments avec tous les attribus prédifinis auquel on rajoute : 
    
    - First_occurence : première d'ajout d'un fichier
    - last_move_date : date du dernier changement de position
    - last_move_date : dernier changement de position
    - last_move_salle : Salle du dernier changement de position
    - Clean date : Si un equipment est jugé clean, la date du dernier ajout est affiché 

In [ ]:
drop table if exists `IT_Equipment_unique_records` ;
create table `IT_Equipment_unique_records`
as 
select * from `IT_Equipment_first_occurence_per_id`
GROUP BY ID_Systeme_Asset ;

In [ ]:
ALTER TABLE IT_Equipment_unique_records
ADD COLUMN first_occurence varchar(255),
ADD COLUMN last_move_salle varchar(255),
ADD COLUMN last_move varchar(255),
ADD COLUMN last_move_date varchar(255),
ADD COLUMN clean_date varchar(255) ;

## Update de la table `IT_Equipment_unique_records`

### First occurence

In [ ]:
UPDATE `IT_Equipment_unique_records` it
INNER JOIN `IT_Equipment_first_occurence_per_id` it2
ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset

set first_occurence = it2.inserted

### Last move

In [ ]:
UPDATE `IT_Equipment_unique_records` it
INNER JOIN `IT_Equipment_last_occurence_position` it2
ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset

set last_move_date = it2.INSERTED ,
    last_move = it2.Reperage_ID ,
    last_move_salle = it2.nom_de_la_salle

### Clean date

In [ ]:
UPDATE `IT_Equipment_unique_records` it
INNER JOIN `IT_Equipment_cleaned_equipment` it2
ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset

set clean_date = it2.last_insert_date


### Query de control qu'il y a aucun mouvement entre les salles

select ID_Systeme_Asset, Nom_de_la_salle, Reperage_ID, last_move_salle, last_move, last_move_date from `IT_Equipment_unique_records`
where Reperage_ID != last_move

select ID_Systeme_Asset, count(distinct(Nom_de_la_salle)) diff_salle, count(distinct(Reperage_ID)) diff_position from `IT_Equipment_records`
GROUP BY ID_Systeme_Asset
ORDER BY count(distinct(Nom_de_la_salle)) desc, count(distinct(Reperage_ID)) desc

## Indicateur

`IT_Equipment_count_equipment_per_salle` : nombre d'equipment par salle par date

In [ ]:
create view IT_Equipment_count_equipment_per_salle
as 
SELECT nom_de_la_salle, INSERTED, count(*) eq_count from `IT_Equipment_records`
GROUP BY nom_de_la_salle, INSERTED

`indicateur_cleaned_per_salle` : nombre d'equipment clean par salle

In [ ]:
DROP TABLE IF EXISTS indicateur_cleaned_per_salle ;# MySQL returned an empty result set (i.e. zero rows).
CREATE TABLE indicateur_cleaned_per_salle
select Nom_de_la_salle, count(*) clean_count from `IT_Equipment_cleaned_equipment`
GROUP BY Nom_de_la_salle

`IT_Equipment_count_equipment_per_salle_first_occurence` : nombre d'equipment par salle pour la première date d'ajout

In [ ]:
drop VIEW if EXISTS IT_Equipment_count_equipment_per_salle_first_occurence;
create view IT_Equipment_count_equipment_per_salle_first_occurence
as 
SELECT nom_de_la_salle,
(SELECT * FROM `date_insertion_records`
 order by source_insert asc
 limit 1 ) date,
count(*) eq_count from `IT_Equipment_records`
where INSERTED = 
				(SELECT * FROM `date_insertion_records`
                order by source_insert asc
                limit 1 )
                
GROUP BY nom_de_la_salle

`IT_Equipment_count_equipment_per_salle_last_occurence` : nombre d'equipment par salle pour la dernière date d'ajout

In [ ]:
drop VIEW if EXISTS IT_Equipment_count_equipment_per_salle_last_occurence;
create view IT_Equipment_count_equipment_per_salle_last_occurence
as 
SELECT nom_de_la_salle,
(SELECT * FROM `date_insertion_records`
 order by source_insert desc
 limit 1 ) date,
count(*) eq_count from `IT_Equipment_records`
where INSERTED = 
				(SELECT * FROM `date_insertion_records`
                order by source_insert desc
                limit 1 )
                
GROUP BY nom_de_la_salle

`indicateur_equipment_per_salle` : indicateur du nombre d'equipment par salle pour la 1er et dernière occurence

In [ ]:
DROP TABLE IF EXISTS indicateur_equipment_per_salle ;# MySQL returned an empty result set (i.e. zero rows).
CREATE TABLE indicateur_equipment_per_salle
SELECT DISTINCT(nom_de_la_salle) from `IT_Equipment_records`
ORDER By Nom_de_la_salle asc ;# 6 rows affected.

### Update indicateur_equipment_per_salle

`indicateur_equipment_per_salle` : Suit le nombre d'equipment unique par salle

     - first_occurence : nb d'equipement dans la première occurence
     - last_occurence : nb d'equipement dans la dernière occurence
     - in_equipment : nb d'equipement entrée
     - out_equipment : nb d'equipement sortie
    
    
    

In [ ]:
ALTER TABLE indicateur_equipment_per_salle
ADD COLUMN first_occurence varchar(255),
ADD COLUMN last_occurence varchar(255),
ADD COLUMN in_equipment varchar(255),
ADD COLUMN out_equipment varchar(255) ;

In [ ]:
UPDATE `indicateur_equipment_per_salle` it 
INNER JOIN `IT_Equipment_count_equipment_per_salle_first_occurence` it2 
ON it.nom_de_la_salle = it2.nom_de_la_salle
SET  it.first_occurence = it2.eq_count ;

UPDATE `indicateur_equipment_per_salle` it 
INNER JOIN `IT_Equipment_count_equipment_per_salle_last_occurence` it2 
ON it.nom_de_la_salle = it2.nom_de_la_salle
SET  it.last_occurence = it2.eq_count ;

alter view hana_query as select ID_Systeme_Asset, nom, Nom_de_la_salle, Nom_de_baie, INSERTED from `IT_Equipment_records` where Nom_de_baie like 'ha%' and INSERTED = '22-02-04'and Nom_de_la_salle = 'P1' 

### 1 et dernier fichier pour controle rapide

In [ ]:
DROP TABLE if EXISTS IT_Equipment_17_06_2022 ; 
CREATE TABLE IT_Equipment_17_06_2022
as
select * from `IT_Equipment_records` where INSERTED = "22-06-17"

In [ ]:
DROP TABLE if EXISTS IT_Equipment_4_02_2022 ; 
CREATE TABLE IT_Equipment_4_02_2022
as
select * from `IT_Equipment_records` where INSERTED = "22-02-04"

`query_meme_nom_resultat_diff` : nom similaire mais asset_id ou id_systeme_asset différents (ce qui est pas normal)

In [ ]:
CREATE view query_meme_nom_resultat_diff
as
select it.nom, it.ASSET_ID, it.reperage_id, it.nom_de_la_salle, it.ID_Systeme_Asset,it2.nom nom_juin, it2.ASSET_ID ASSET_ID_juin, it2.reperage_id reperage_id_juin, it2.nom_de_la_salle nom_de_la_salle_juin, it2.ID_Systeme_Asset ID_Systeme_Asset_juin
from `IT_Equipment_4_02_2022` it
INNER JOIN `IT_Equipment_17_06_2022` it2 
ON it.nom = it2.nom

where it.ASSET_ID != it2.ASSET_ID or it.ID_Systeme_Asset != it2.ID_Systeme_Asset

`IT_Equipment_unique_name` : table des noms pour le fichier it_equipment du 17/06/22

In [ ]:
drop view if EXISTS IT_Equipment_unique_name ;
create view IT_Equipment_unique_name
as
SELECT nom, count(nom) number_row FROM `IT_Equipment_17_06_2022`

GROUP BY nom
HAVING number_row = 1
ORDER by count(nom) DESC

`IT_Equipment_move_salle_name_match` : mouvement de salle par rapport aux nom unique

In [ ]:
drop view if exists IT_Equipment_move_salle_name_match ;
create VIEW IT_Equipment_move_salle_name_match
as
select it.nom, it.ASSET_ID, it.reperage_id, it.nom_de_la_salle, it.ID_Systeme_Asset,it2.nom nom_juin, it2.ASSET_ID ASSET_ID_juin, it2.reperage_id reperage_id_juin, it2.nom_de_la_salle nom_de_la_salle_juin, it2.ID_Systeme_Asset ID_Systeme_Asset_juin
FROM `IT_Equipment_unique_name` it3

INNER JOIN `IT_Equipment_4_02_2022` it
ON it3.nom = it.nom
INNER JOIN `IT_Equipment_17_06_2022` it2 
ON it3.nom = it2.nom

where it.nom_de_la_salle != it2.nom_de_la_salle --  or it.reperage_id != it2.reperage_id

`IT_Equipment_non_unique_nom` : equipments avec un nom non unique

In [ ]:
drop VIEW if EXISTS IT_Equipment_non_unique_nom ;
create VIEW IT_Equipment_non_unique_nom
as
SELECT nom, count(nom) number_row, Type FROM `IT_Equipment_17_06_2022`

GROUP BY nom
HAVING number_row != 1
ORDER by count(nom) DESC

### Equipement mise à jour sous un ID_systeme_asset différents

`IT_Equipment_move_no_asset` : Si un id_systeme_asset n'est pas dispo dans le dernier fichier csv (table des élements clean`IT_Equipment_cleaned_equipment`), on procède à un check des noms. S'il y a concordence des noms dans le dernier fichier c'est que id_systeme_asset a été modifié

In [ ]:
drop view if exists IT_Equipment_move_no_asset ;
create view IT_Equipment_move_no_asset
as 
select it.nom nom_clean, it2. nom nom_new, 
it.ID_Systeme_Asset asset_clean,  it2.ID_Systeme_Asset asset_new,
it.asset_id asset_id_clean,  it2.asset_id asset_id_new,
it.nom_de_la_salle salle_clean, it.Reperage_ID position_clean,
it2.nom_de_la_salle salle_new, it2.Reperage_ID position_new,
it.last_insert_date date_clean, it2.inserted date_new
from `IT_Equipment_cleaned_equipment` it 
INNER JOIN `IT_Equipment_records` it2
ON it.nom = it2.nom

WHERE it2.inserted =(
    				SELECT * FROM `last_date_insertion`
    				)